<a href="https://colab.research.google.com/github/hmobley/liveproject_water/blob/master/Milestone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install sentinelsat
%pip install rasterio

In [ ]:
import rasterio
import glob
import os

In [ ]:
# load an image with rastario

In [ ]:
# display the image

In [ ]:
# manipulate the image with tensorflow

In [ ]:
# display the image

In [ ]:
# clean up files